# **pytorch 데이터다루기**

# **1.텐서 다루기 기초**

In [ ]:
import torch
import numpy as np
import pandas as pd

## **(1) 텐서 만들기 : torch.tensor()**


* 리스트, 넘파이로 부터

In [ ]:
# 리스트
data = [[1, 2],[3, 4],[5, 6]]
x_data = torch.tensor(data)
x_data

In [ ]:
# 넘파이 어레이
np_array = np.array(data)
x_np = torch.tensor(np_array)
x_np

* 데이터프레임은 지원하지 않음.

In [ ]:
df = pd.DataFrame(data)
display(df)

x_df = torch.tensor(df)  # 오류 발생
x_df

------------------------------------------------------------------------


## **(2) 텐서의 속성**


* 텐서의 속성 확인하기

In [ ]:
print(x_np)
print(f"Shape : {x_np.shape}")
print(f"Datatype : {x_np.dtype}")
print(f"Device : {x_np.device}")

* 생성 시 데이터타입 지정
    * 보통, 다양한 연산을 위해서 dtype = torch.float32 로 지정

In [ ]:
x_np = torch.tensor(np_array, dtype = torch.float32)

print(x_np)
print(f"Shape : {x_np.shape}")
print(f"Datatype : {x_np.dtype}")
print(f"Device : {x_np.device}")

------------------------------------------------------------------------


## **(3) Operations**


* 텐서 작업을 디바이스(device)에 할당
    * cpu, gpu

In [ ]:
# GPU가 사용 가능한 경우에는 GPU에, 그렇지 않은 경우에는 CPU에 할당
if torch.cuda.is_available():
    tensor = x_np.to("cuda")
else:
    tensor = x_np.to("cpu")

# 텐서가 어느 디바이스에 할당되어 있는지 확인
print(tensor.device)

* 넘파이처럼 다루기


In [ ]:
np_array = np.array(data)
x_np = torch.tensor(np_array)
print(x_np)
print('='*30)

# 열, 행 인덱스로 조회
print(f"첫 행: {x_np[0]}")
print(f"첫 열: {x_np[:, 0]}")
print(f"마지막 열: {x_np[:, -1]}")
print('='*30)

# 값 수정
x_np[:,1] = 0
print(x_np)

* 결합 : torch.cat

In [ ]:
np_array = np.array(data)
x_np = torch.tensor(np_array, dtype = torch.float32)
x_np

In [ ]:
t0 = torch.cat([x_np, x_np], dim=0)
print(t0)

In [ ]:
t1 = torch.cat([x_np, x_np], dim=1)
print(t1)

## **(4) 연산**


* 집계함수

In [ ]:
print(x_np)
print(x_np.sum())
print(x_np.mean())
print(x_np.std())

* 행렬 곱

In [ ]:
y1 = x_np @ x_np.T
y2 = x_np.matmul(x_np.T)

print(y1)
print(y2)

In [ ]:
y1 = x_np @ x_np.T
y2 = x_np.T @ x_np

print(y1)
print(y2)

* 요소 곱(element-wise product)

In [ ]:
z1 = x_np * x_np
z2 = x_np.mul(x_np)

print(z1)
print(z2)

* 단일 요소 텐서  
단일 요소 텐서로 부터 값을 뽑아서 사용하려면, .item()

In [ ]:
agg = x_np.mean()
print(agg, type(agg))
print('='*30)

agg_item = agg.item() # .item()은 단일요소 텐서에서 사용 가능
print(agg_item, type(agg_item))

# **2.dataset 다루기**

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

### **(1) 데이터 준비**


In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/advertising.csv'
data = pd.read_csv(path)
data = data[:10]  # 10건만 저장하기.
target = 'Sales'
x = data.drop(target, axis=1)
y = data.loc[:, target]

print(x.shape, y.shape)

### **(2) 텐서로 변환**

* torch.tensor(numpy array, )
    * numpy array를 입력으로 받음

In [ ]:
# pandas dataframe, series를 numpy array로 변환
# 현재 자료형 타입
print(type(x), type(y))
print('='*50)

# 넘파이로 바꾸기
x_np = x.values
y_np = y.values

print(type(x_np), type(y_np))
print(x_np.shape, y_np.shape)
print('='*50)

# y도 2차원 구조가 필요.
y_np = y_np.reshape(10, 1)  # 10행 1열 구조
print(x_np.shape, y_np.shape)

In [ ]:
# 텐서로 변환
x_tensor = torch.tensor(x_np, dtype=torch.float32)
y_tensor = torch.tensor(y_np, dtype=torch.float32)

print(x_tensor.shape, y_tensor.shape)

* 텐서 내용 열어보기

In [ ]:
x_tensor[:5]

In [ ]:
y_tensor[:5]

### **(3) Data Loader 준비**
* Data Loader는 텐서 데이터셋을 입력으로 받음
    * 그래서, 텐서x, y를 하나의 **텐서 데이터셋**으로 합치기
    * 그 다음 DataLoader 생성

In [ ]:
# TensorDataset 생성 : 텐서 데이터셋으로 합치기
dataset = TensorDataset(x_tensor, y_tensor)

print(dataset[:5])

In [ ]:
# 데이터 로더 생성
data_loader = DataLoader(dataset, batch_size = 3, shuffle = True)

* 데이터 로더로 부터 데이터 추출하기

In [ ]:
# 데이터 로더로 부터 첫번째 배치 데이터 로딩
for (x_b, y_b) in data_loader:
    print(x_b)
    print(y_b)
    break

In [ ]:
# 데이터 로더로부터 배치 단위로 데이터 로딩
for batch_id, (x_b, y_b) in enumerate(data_loader):
    print(f'batch_id : {batch_id}')
    print(x_b)
    print(y_b)
    print('='* 50)

# **3.device**

* 텐서를 넘파이로 변환하려면,
    * device 할당된 텐서
        * cpu --> .numpy()
        * gpu --> .cpu().numpy()
    * device 할당 안된 텐서 : .numpy()

In [ ]:
x_tensor.numpy()